###Deltalake & Lakehouse Optimization Usecases

![](/Workspace/Users/infoblisstech@gmail.com/databricks-code-repo/5_all_databricks_workouts/DELTA OPTIMIZATIONS.png)

####1. Handling Data Skew & Query Performance (Optimize & Z-Order)
Scenario: The analytics team reports that queries filtering silver_shipments by source_city and shipment_date are becoming slow as data volume grows.

Task: Run the OPTIMIZE command with ZORDER on the silver_shipments table to co-locate related data in the same files.

Outcome:
Why did we choose source_city and shipment_date for Z-Ordering instead of shipment_id? Think about high cardinality vs. query filtering

In [0]:
%sql
use prodcatalog_1.logistics

In [0]:
%sql
DESCRIBE DETAIL silver_shipments;

In [0]:
%sql
OPTIMIZE silver_shipments;

In [0]:
%sql
DESCRIBE HISTORY silver_shipments;

In [0]:
%sql
OPTIMIZE silver_shipments
ZORDER BY (source_city,shipment_date);

In [0]:
%sql
DESCRIBE HISTORY silver_shipments;

In [0]:
%sql
SELECT * FROM silver_shipments;

#### 2. Speeding up Regional Queries (Partition Pruning)
Scenario: The dashboard team reports that queries filtering for orgin_hub_city with "New York" shipments from the gold_core_curated_tbl table are scanning the entire dataset (Terabytes of data), even though New York is only 5% of the data. This is racking up compute costs.

Task: Re-create the gold_core_curated_tbl table partitioned by orgin_hub_city. Run a query filtering for one city to demonstrate "Partition Pruning" (where Spark skips files that don't match the filter).

Outcome: Verify the partition filtering is applied or not, by performing explain plan, check for the PartitionFilters in the output.

In [0]:
%sql
USE prodcatalog_1.logistics;

CREATE OR REPLACE TABLE origin_hub_city_part 
USING DELTA
PARTITIONED BY (origin_hub_city) 
AS
SELECT * FROM gold_core_curated_tbl;



In [0]:
%sql
SHOW PARTITIONS origin_hub_city_part;

In [0]:
%sql
EXPLAIN SELECT * FROM origin_hub_city_part WHERE origin_hub_city='Mumbai';

#### 3. Storage Cost Savings (Vacuum)
Scenario: Your Project pipeline runs every hour, creating many small files and obsolete versions of data. Your storage costs are rising. You need to clean up files that are no longer needed for time travel.

Task: Execute a Vacuum command to remove data files older than the retention threshold.

Outcome: Performance improvement, cost saving, best practices.

Observation: Perform the describe history and find whether vacuum is completed.

In [0]:
%sql
VACUUM workspace.default.mobile_os_usage1 RETAIN 168 HOURS;

In [0]:
%sql
DESCRIBE HISTORY workspace.default.mobile_os_usage1;

####4. Modern Data Layout (Liquid Clustering)
Scenario: You are redesigning the silver_shipments table. You want to avoid the "small files" problem and need a flexible layout that adapts to changing query patterns automatically without rewriting the table.

Task: Re-create the silver_shipments table using Liquid Clustering on the shipment_id column.

Outcome: Liquid Clustering over traditional partitioning when the cardinality of shipment_id is very high.

In [0]:
%sql
CREATE OR REPLACE TABLE silver_shipment
USING DELTA
CLUSTER BY (shipment_id) AS
SELECT * FROM silver_shipments;


In [0]:
%sql
DESCRIBE HISTORY silver_shipment;

#### 5. Cost Efficient Environment Cloning (Shallow Clone)
Scenario: The QA team needs to test an update on the gold_core_curated_tbl table. The table is 5TB in size. You cannot afford to duplicate the storage cost just for a test and the update should not affect the copied table.

Task: Create a Shallow Clone of the gold table for the QA team.

Outcome: If we delete records from the source table (gold_core_curated_tbl), will the QA table (gold_core_curated_tbl_qa) be affected? Why or why not?

In [0]:
%sql
CREATE OR REPLACE TABLE gold_core_curated_tbl_qa
SHALLOW CLONE gold_core_curated_tbl;

#### 6. Disaster Recovery (Time Travel & Restore)
Scenario: A junior data engineer accidentally ran a logic error that corrupted the gold_core_curated_tbl table 15 minutes ago. You need to revert the table to its previous state immediately.

Task: Use Delta Lake's Restore feature to roll back the table.

Outcome:What is the difference between querying with VERSION AS OF (Time Travel) and running RESTORE?

In [0]:
%sql
SELECT * FROM gold_core_curated_tbl VERSION AS OF 1;
SELECT * FROM gold_core_curated_tbl TIMESTAMP AS OF '2025-01-15 10:30:00';

RESTORE TABLE gold_core_curated_tbl TO VERSION AS OF 1;
RESTORE TABLE gold_core_curated_tbl TO TIMESTAMP AS OF '2025-01-15 10:30:00'; 


#Outcome
VERSION AS OF is for safely reading old data, while RESTORE permanently rolls the table back by creating a new Delta version.